- Fine-tune multi-lingual - with both Hindi and Nepali
- Fine tune with Hindi run inference on Nepali 
- Fine tune with Nepali run inference on Hindi
- Fine tune with Nepali run inference on Nepali
- Fine tune with Hindi run inference on Hindi

In [1]:
from kaggle_secrets import UserSecretsClient


In [2]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [3]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 109 (delta 53), reused 59 (delta 15), pack-reused 0
Receiving objects: 100% (109/109), 27.41 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [ ]:
import os
import wandb

os.environ["WANDB_PROJECT"]="llama-finetune-nepali"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_token = user_secrets.get_secret("WANDB_KEY")
wandb.login(key= wandb_token)

In [ ]:
from datasets import load_dataset, load_metric, concatenate_datasets
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import re

In [ ]:
split_ratio = 0.2
seed = 42

In [ ]:
dataset_name = "CohereForAI/aya_dataset"
aya_dataset = load_dataset(dataset_name)
dataset_type = "aya"

In [ ]:
lang_multi = "Nepali"
aya_nepali_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)
aya_nepali_train

In [ ]:
lang_multi = "Hindi"
aya_hindi_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)
aya_hindi_train

In [ ]:
aya_nepali_train = aya_nepali_train.rename_columns(
            {"inputs": "input", "targets": "output"})
new_column_data = [""] * len(aya_nepali_train)  # Create a list with the same length as the dataset
aya_nepali_train = aya_nepali_train.add_column("instruction", new_column_data)
aya_nepali_train = aya_nepali_train.remove_columns(['language', 'language_code', 'annotation_type', 'user_id'])
aya_nepali_train

In [ ]:
aya_hindi_train = aya_hindi_train.rename_columns(
            {"inputs": "input", "targets": "output"})
new_column_data = [""] * len(aya_hindi_train)
aya_hindi_train = aya_hindi_train.add_column("instruction", new_column_data)
aya_hindi_train = aya_hindi_train.remove_columns(['language', 'language_code', 'annotation_type', 'user_id'])
aya_hindi_train

In [ ]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"

In [ ]:
alpaca_nepali_train, _ = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)
alpaca_nepali_train

In [ ]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"

In [ ]:
alpaca_hindi_train, _ = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)
alpaca_hindi_train

In [ ]:
nepali_train = concatenate_datasets([aya_nepali_train, alpaca_nepali_train])
nepali_train = nepali_train.shuffle(seed=42)
nepali_train

In [ ]:
hindi_train = concatenate_datasets([aya_hindi_train, alpaca_hindi_train])
hindi_train = hindi_train.shuffle(seed=42)
hindi_train

In [ ]:
# both_train = concatenate_datasets([nepali_train, hindi_train])
# both_train = both_train.shuffle(seed=42)
# both_train

In [ ]:
#Nepali
prompt = """Below is an instruction in Nepali that describes a task, paired with an input also in Nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

train_data = nepali_train
train_data = train_data.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
#         max_steps = 1000,                #### Setting max_steps for 1000. (1-1000)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        save_steps=500,                 ### Checkpoint will be save after every 500 steps
        optim = "adamw_8bit",
        num_train_epochs=2,
        weight_decay = 0.01,
        report_to="wandb", 
        run_name="llama3_alpaca_nep", 
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_alpaca_nep",   # Saving the checkpoints to outputs folder
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
wandb.finish()